In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#create random initial spins in an NxN array?
#Need to take into account the position of the spins

#Pick one random spin and flip it
#Calculate energy difference between the old and the new state
#If E_new > E_old: keep new state
#If E_new < E_old: calculate q = p(x')/p(x).
#Draw random number: if random number lower than q: accept new state

In [3]:
def initial_spins(N):
    lattice = np.random.choice([1], size=(N,N))
    
    return lattice



# def system_energy(lattice, coupling_constant):
#     L = len(lattice[0,:])
#     ham = 0
#     for i in range(L):
#         for j in range(L):
#             h = 0
#             h = lattice[i,j]*lattice[(i-1)%L,j-1] + lattice[i,j]*lattice[(i-1)%L,(j+1)%L] + lattice[i,j]*lattice[(i+1)%L,(j+1)%L] + lattice[i,j]*lattice[(i+1)%L,(j-1)%L]
#             ham += h
#     total_energy = -coupling_constant*ham
#     return total_energy


def system_energy(lattice, J):
#     this method is way faster the the double for loop

    energy = -J * np.sum(lattice * (np.roll(lattice, -1, axis=0) + np.roll(lattice, -1, axis=1)))
    return energy




def energy_difference(old_lattice, new_lattice):
    delta_energy = total_hamiltonian(new_lattice) - total_hamiltonian(old_lattice)
    return delta_energy


def monte_carlo(lattice, coupling_constant, temperature):
    beta = 1/temperature
    size = len(lattice[0,:])
    random_x, random_y = np.random.randint(size, size=2)
    delta_E = 2 * coupling_constant * lattice[random_x, random_y] * (lattice[(random_x-1)%size, random_y] 
                                                              + lattice[(random_x+1)%size, random_y] 
                                                              + lattice[random_x, (random_y-1)%size] 
                                                              + lattice[random_x, (random_y+1)%size])
    if delta_E <= 0:
        lattice[random_x, random_y] *= -1
        return lattice, delta_E
    if np.random.rand() < np.exp(-delta_E * beta):
        lattice[random_x, random_y] *= -1
        return lattice, delta_E
    else:
        return lattice, 0

def show_snapshot(lattice, timestep):
    
    plt.imshow(lattice, cmap='binary')
    plt.title(f'Timestep {timestep}')
    plt.show()


def ising_simulation(size, coupling_constant, temperature, timesteps):
#     two metropolis algorithm starting from the same initial lattice
    lattice = initial_spins(size)
    lattice2 = np.copy(lattice)
    energies = []
    energies2 = []
    magnetizations = []
    magnetizations2 = []
    t_to_eq = 10000
    for i in range(timesteps + t_to_eq):
        lattice, delta_E = monte_carlo(lattice, coupling_constant, temperature) 
        lattice2, delta_E2 = monte_carlo(lattice2, coupling_constant, temperature)
        if i >= t_to_eq:
            energies.append(system_energy(lattice, coupling_constant)/size**2)
            magnetizations.append(np.sum(lattice)/size**2)
            energies2.append(system_energy(lattice2, coupling_constant)/size**2)
            magnetizations2.append(np.sum(lattice2)/size**2)
#             if i%1000 == 0:
#                 show_snapshot(lattice, i-t_to_eq)
                
    return np.array(energies), np.array(magnetizations), np.array(energies2), np.array(magnetizations2)

def auto_correlation(magnetization, timestep):
    xi = np.zeros(timestep)
    for t in range(timestep):
        xi_element = 0
        for t_prime in range(timestep-t):
            xi_element += 1/(timestep-t)*magnetization[t_prime]*magnetization[t_prime+t] -( 1/(timestep-t)*magnetization[t_prime] * 1/(timestep-t)*magnetization[t_prime+t] )
        xi[t] = xi_element
    return xi

In [4]:
#Define constants
n_spins = 10
coupling_constant = 1
temperature = 3
n_steps = 50000

In [5]:
# Run the simulation
energies, magnetizations, energies2, magnetizations2 = ising_simulation(n_spins, coupling_constant, temperature, n_steps)

In [ ]:
chi = auto_correlation(magnetizations, n_steps)

In [ ]:
# Plot the results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.plot(energies)
ax1.plot(energies2)
ax1.set_xlabel("Step")
ax1.set_ylabel("Energy")
ax2.plot(chi)
# ax2.plot(magnetizations2)
ax2.set_xlabel("Step")
ax2.set_ylabel("Correlation")
plt.show()